In [3]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb

from lightgbm import LGBMModel,LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import ClusterCentroids,RandomUnderSampler,AllKNN
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,roc_auc_score
import warnings
warnings.simplefilter("ignore")

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
test_trans=pd.read_csv('/content/drive/My Drive/test_transaction.csv')

In [5]:
train_trans = pd.read_csv('/content/drive/My Drive/train_transaction.csv')

In [6]:
test_id = pd.read_csv('/content/drive/My Drive/test_identity.csv')

In [7]:
train_id = pd.read_csv('/content/drive/My Drive/train_identity.csv')

In [8]:
print('Train_identity: ', train_id.shape)
print('Train_transaction: ', train_trans.shape)
print('Test_identity: ',test_id.shape)
print('Test_transaction: ', test_trans.shape)

Train_identity:  (144233, 41)
Train_transaction:  (590540, 394)
Test_identity:  (141907, 41)
Test_transaction:  (506691, 393)


In [9]:
train = pd.merge(train_id,train_trans,left_on='TransactionID',right_on='TransactionID',how='right')
test = pd.merge(test_id,test_trans,left_on='TransactionID',right_on='TransactionID',how='right')
train.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
del train_id
del train_trans
del test_id
del test_trans
gc.collect()

0

In [11]:
def check_value(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% Missing'})
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% Missing', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
        return mz_table

In [12]:
Missing_values_table = check_value(train)
train_null_tables= Missing_values_table.loc[Missing_values_table['% Missing'] > 90]
train_null_tables['Unique'] = [train[col].nunique() for col in train_null_tables.index ]
print (train_null_tables)

Your selected dataframe has 434 columns and 590540 Rows.
There are 414 columns that have missing values.
       Zero Values  Missing Values  % Missing Data Type  Unique
id_24            0          585793       99.2   float64      12
id_25            0          585408       99.1   float64     341
id_07          409          585385       99.1   float64      84
id_08          261          585385       99.1   float64      94
id_21            0          585381       99.1   float64     490
id_26            0          585377       99.1   float64      95
id_23            0          585371       99.1    object       3
id_22            0          585371       99.1   float64      25
id_27            0          585371       99.1    object       2
dist2         3519          552913       93.6   float64    1751
D7           21135          551623       93.4   float64     597
id_18            0          545427       92.4   float64      18


In [13]:
id_cate = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18','id_19', 'id_20', 'id_21', 'id_22', 'id_23','id_24', 'id_25','id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32','id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
id_col=[]
for col in train[id_cate]:
    id_col.append(train[col].mode().iloc[0])
    train[col] = train[col].fillna(train[col].mode().iloc[0])
train[id_cate].head()

,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
0,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F
1,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F
2,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F
3,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F
4,NotFound,52.0,-480.0,New,NotFound,166.0,15.0,542.0,144.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T


In [14]:
test_id_cate = ['id-12', 'id-13', 'id-14', 'id-15', 'id-16', 'id-17', 'id-18','id-19', 'id-20', 'id-21', 'id-22', 'id-23','id-24', 'id-25','id-26', 'id-27', 'id-28', 'id-29', 'id-30', 'id-31', 'id-32','id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38']
i = 0
for col in test[test_id_cate]:
    test[col] = test[col].fillna(id_col[i])
    i += 1
test[test_id_cate].head()

,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38
0,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F
1,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F
2,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F
3,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F
4,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F


In [15]:
train_y = train['isFraud']
train = train.drop(columns=['isFraud'])

In [16]:
fill_col =[]
for col in train:
  if train[col].dtype != 'object':
    #  print(train[col].mean())
    fill_col.append(train[col].mean())
    train[col] = train[col].fillna(train[col].mean())
train.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
1,2987001,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
2,2987002,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
3,2987003,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
4,2987004,0.000000,70787.000000,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,100.000000,NotFound,52.0,-480.0,New,NotFound,166.0,15.0,542.0,144.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [17]:
i = 0
for col in test:
  if test[col].dtype != 'object':
    test[col] = test[col].fillna(fill_col[i])
    i += 1
test.head()

,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,id-10,id-11,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,47.950001,0.000000,0.000000,47.950001,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
1,3663550,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,280.000000,77.000000,0.000000,280.000000,0.000000,77.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
2,3663551,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,1321.000000,0.000000,0.000000,1058.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,263.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
3,3663552,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
4,3663553,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,67.949997,183.850006,67.949997,67.949997,183.850006,67.949997,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882


In [18]:
cate_col=[]
for col in train:
  if train[col].dtype == 'object':
    cate_col.append(train[col].mode().iloc[0])
    train[col] = train[col].fillna(train[col].mode().iloc[0])
train.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
1,2987001,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
2,2987002,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
3,2987003,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,...,0.0,0.0,0.0,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
4,2987004,0.000000,70787.000000,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,100.000000,NotFound,52.0,-480.0,New,NotFound,166.0,15.0,542.0,144.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [19]:
i = 0
for col in test:
  if test[col].dtype == 'object':
    test[col] = test[col].fillna(cate_col[i])
    i += 1
test.head()

,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,id-10,id-11,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,47.950001,0.000000,0.000000,47.950001,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
1,3663550,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,280.000000,77.000000,0.000000,280.000000,0.000000,77.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
2,3663551,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,1321.000000,0.000000,0.000000,1058.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,263.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
3,3663552,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
4,3663553,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,NotFound,52.0,-300.0,Found,Found,166.0,15.0,266.0,507.0,252.0,14.0,IP_PROXY:TRANSPARENT,11.0,321.0,161.0,Found,Found,Found,Windows 10,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,...,0.0,0.0,0.0,0.0,0.0,1.0,67.949997,183.850006,67.949997,67.949997,183.850006,67.949997,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882


In [20]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [21]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 649.91 Mb (66.8% reduction)
Mem. usage decreased to 565.37 Mb (66.3% reduction)


In [22]:
cat_col=[]
for col in train:
  if train[col].dtype == 'object':
    cat_col.append(col)
for col in cat_col:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col].astype(str).values)
train.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,-10.171875,174716.578125,0.060181,-0.058929,1.615234,-6.699219,13.289062,-38.59375,0.091003,-0.301025,99.75,1,52.0,-300.0,0,0,166.0,15.0,266.0,507.0,252.0,14.0,2,11.0,321.0,161.0,0,0,0,42,47,24.0,132,3,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,6.21875,13.101562,9.1875,0.058502,0.851074,0.296631,0.33667,1.3125,0.775879,721.741882,1375.783691,1014.622803,9.804688,59.15625,28.53125,55.352421,151.160538,100.700882
1,2987001,-10.171875,174716.578125,0.060181,-0.058929,1.615234,-6.699219,13.289062,-38.59375,0.091003,-0.301025,99.75,1,52.0,-300.0,0,0,166.0,15.0,266.0,507.0,252.0,14.0,2,11.0,321.0,161.0,0,0,0,42,47,24.0,132,3,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.21875,13.101562,9.1875,0.058502,0.851074,0.296631,0.33667,1.3125,0.775879,721.741882,1375.783691,1014.622803,9.804688,59.15625,28.53125,55.352421,151.160538,100.700882
2,2987002,-10.171875,174716.578125,0.060181,-0.058929,1.615234,-6.699219,13.289062,-38.59375,0.091003,-0.301025,99.75,1,52.0,-300.0,0,0,166.0,15.0,266.0,507.0,252.0,14.0,2,11.0,321.0,161.0,0,0,0,42,47,24.0,132,3,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.21875,13.101562,9.1875,0.058502,0.851074,0.296631,0.33667,1.3125,0.775879,721.741882,1375.783691,1014.622803,9.804688,59.15625,28.53125,55.352421,151.160538,100.700882
3,2987003,-10.171875,174716.578125,0.060181,-0.058929,1.615234,-6.699219,13.289062,-38.59375,0.091003,-0.301025,99.75,1,52.0,-300.0,0,0,166.0,15.0,266.0,507.0,252.0,14.0,2,11.0,321.0,161.0,0,0,0,42,47,24.0,132,3,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,6.21875,13.101562,9.1875,0.058502,0.851074,0.296631,0.33667,1.3125,0.775879,721.741882,1375.783691,1014.622803,9.804688,59.15625,28.53125,55.352421,151.160538,100.700882
4,2987004,0.000000,70787.000000,0.060181,-0.058929,1.615234,-6.699219,13.289062,-38.59375,0.091003,-0.301025,100.00,1,52.0,-480.0,1,1,166.0,15.0,542.0,144.0,252.0,14.0,2,11.0,321.0,161.0,0,1,1,7,123,32.0,164,3,1,0,1,1,1,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000


In [23]:
test_col=[]
for col in test:
  if test[col].dtype == 'object':
    test_col.append(col)
for col in test_col:
  le = LabelEncoder()
  test[col] = le.fit_transform(test[col].astype(str).values)
test.head()

,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,id-10,id-11,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,-10.171875,174716.578125,0.060181,-0.058929,1.615234,-6.699219,13.289062,-38.59375,0.091003,-0.301025,99.75,1,52.0,-300.0,0,0,166.0,15.0,266.0,507.0,252.0,14.0,2,11.0,321.0,161.0,0,0,0,48,32,24.0,171,1,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,47.950001,0.000000,0.0000,47.950001,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,6.21875,13.101562,9.1875,0.058502,0.851074,0.296631,0.33667,1.3125,0.775879,721.741882,1375.783691,1014.622803,9.804688,59.15625,28.53125,55.352421,151.160538,100.700882
1,3663550,-10.171875,174716.578125,0.060181,-0.058929,1.615234,-6.699219,13.289062,-38.59375,0.091003,-0.301025,99.75,1,52.0,-300.0,0,0,166.0,15.0,266.0,507.0,252.0,14.0,2,11.0,321.0,161.0,0,0,0,48,32,24.0,171,1,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,280.000000,77.000000,0.0000,280.000000,0.0000,77.000000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,6.21875,13.101562,9.1875,0.058502,0.851074,0.296631,0.33667,1.3125,0.775879,721.741882,1375.783691,1014.622803,9.804688,59.15625,28.53125,55.352421,151.160538,100.700882
2,3663551,-10.171875,174716.578125,0.060181,-0.058929,1.615234,-6.699219,13.289062,-38.59375,0.091003,-0.301025,99.75,1,52.0,-300.0,0,0,166.0,15.0,266.0,507.0,252.0,14.0,2,11.0,321.0,161.0,0,0,0,48,32,24.0,171,1,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,1321.000000,0.000000,0.0000,1058.000000,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0,263.0,0.0,6.21875,13.101562,9.1875,0.058502,0.851074,0.296631,0.33667,1.3125,0.775879,721.741882,1375.783691,1014.622803,9.804688,59.15625,28.53125,55.352421,151.160538,100.700882
3,3663552,-10.171875,174716.578125,0.060181,-0.058929,1.615234,-6.699219,13.289062,-38.59375,0.091003,-0.301025,99.75,1,52.0,-300.0,0,0,166.0,15.0,266.0,507.0,252.0,14.0,2,11.0,321.0,161.0,0,0,0,48,32,24.0,171,1,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,282.5000,282.500,282.5000,0.0,0.0,0.0,0.0,0.0,0.0,6.21875,13.101562,9.1875,0.058502,0.851074,0.296631,0.33667,1.3125,0.775879,721.741882,1375.783691,1014.622803,9.804688,59.15625,28.53125,55.352421,151.160538,100.700882
4,3663553,-10.171875,174716.578125,0.060181,-0.058929,1.615234,-6.699219,13.289062,-38.59375,0.091003,-0.301025,99.75,1,52.0,-300.0,0,0,166.0,15.0,266.0,507.0,252.0,14.0,2,11.0,321.0,161.0,0,0,0,48,32,24.0,171,1,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,67.949997,183.850006,67.949997,67.9375,183.850006,67.9375,67.949997,67.9375,183.875,67.9375,0.0,0.0,0.0,0.0,0.0,0.0,6.21875,13.101562,9.1875,0.058502,0.851074,0.296631,0.33667,1.3125,0.775879,721.741882,1375.783691,1014.622803,9.804688,59.15625,28.53125,55.352421,151.160538,100.700882


In [24]:
train_x = train

In [25]:
del cat_col
del Missing_values_table
del train_null_tables
del cate_col
del fill_col
del train
del test_col
del test_id_cate
gc.collect()

272

In [27]:
train_x, train_y = RandomUnderSampler().fit_resample(train_x, train_y)


In [28]:
X_train,X_test,y_train,y_test = train_test_split(train_x,train_y,shuffle = True,random_state = 255,test_size = 0.2)
del train_x
gc.collect()

387

In [29]:
model = DecisionTreeClassifier(random_state=1)
model.fit(X_train,y_train)
test_y = model.predict(X_test)

In [30]:
print('Accuracy score:',accuracy_score(y_test,test_y))
print('F1 score:',f1_score(y_test,test_y))
print('Area Under the Receiver Operating Characteristic Curve:',roc_auc_score(y_test,test_y))
print('Confusion maxtrix',confusion_matrix(y_test,test_y))

Accuracy score: 0.798935398015969
F1 score: 0.8000481231953802
Area Under the Receiver Operating Characteristic Curve: 0.7990444950534802
Confusion maxtrix [[3279  890]
 [ 772 3325]]


In [31]:
forest = RandomForestClassifier(criterion='entropy',random_state=50,n_estimators=100,)
forest.fit(X_train,y_train)
test_y = forest.predict(X_test)

In [32]:
print('Accuracy score:',accuracy_score(y_test,test_y))
print('F1 score:',f1_score(y_test,test_y))
print('Area Under the Receiver Operating Characteristic Curve:',roc_auc_score(y_test,test_y))
print('Confusion maxtrix',confusion_matrix(y_test,test_y))

Accuracy score: 0.8492620372610694
F1 score: 0.8447545477199103
Area Under the Receiver Operating Characteristic Curve: 0.8490735546892861
Confusion maxtrix [[3630  539]
 [ 707 3390]]


In [33]:
submission = pd.read_csv('/content/drive/My Drive/sample_submission.csv')

In [36]:
y_preds1 = model.predict_proba(test)
subm_rf = pd.DataFrame({
    'TransactionID' : submission.TransactionID,
    'isFraud' : y_preds1[:,1]
})
subm_rf.head()

,TransactionID,isFraud
0,3663549,0.0
1,3663550,0.0
2,3663551,1.0
3,3663552,0.0
4,3663553,1.0


In [34]:
y_preds = forest.predict_proba(test)
submission_rf = pd.DataFrame({
    'TransactionID' : submission.TransactionID,
    'isFraud' : y_preds[:,1]
})
submission_rf.head()

,TransactionID,isFraud
0,3663549,0.21
1,3663550,0.18
2,3663551,0.49
3,3663552,0.16
4,3663553,0.27


In [35]:
submission_rf.to_csv('random_forest.csv',index=False)